In [ ]:
#YANK BP
import textwrap, yaml, yank, sys, os, glob, shutil, openmm, rdkit
#import nglview
import openmmtools as mmtools
import yank.utils
import numpy as np
from yank.experiment import YankLoader, YankDumper
import netCDF4 as nc
import matplotlib.pyplot as plt
from mdtraj.formats.dcd import DCDTrajectoryFile
import mdtraj as md
import MDAnalysis as mda
#OPENFF
from openff.units import Quantity, unit
from openmm import unit as openmm_unit
from pdbfixer import PDBFixer
from openff.toolkit import ForceField, Molecule, Topology
from openbabel import openbabel

from ybbp_utils import *

In [ ]:
# Split Crystal into protein and ligand pdbs based on selection strings
receptor_path, ligand_path = seperate_crys_using_MDA('3mxf.pdb', 'resname JQ1')

In [ ]:
# Convert ligand pdb above to an sdf with openbabel
ligand_path = obabel_conversion((ligand_path,'ligand.sdf'))

In [ ]:
# Protonate the receptor (ph7)
receptor_path = protonate_with_pdb2pqr(receptor_path, 'receptor_H.pdb')

In [ ]:
# Load a molecule from a SDF file
ligand = Molecule.from_file(ligand_path)

# Print out a SMILES code for the ligand
print(ligand.to_smiles(explicit_hydrogens=False))

In [ ]:
#RECEPTOR MUST ALREADY BE PROTONATED FOR THIS STEP
fixer = PDBFixer(receptor_path)
fixer.addSolvent(padding=1.0 * openmm_unit.nanometer, ionicStrength=0.15 * openmm_unit.molar)
# ADD PDBFixer hydrogens and parsing crystal structures (Hydrogens with pdb2pqr30 at the moment)

with open("receptor_H_solvated.pdb", "w") as f:
    openmm.app.PDBFile.writeFile(fixer.topology, fixer.positions, f)


In [ ]:
# Openff function to add the ligand and remove clashing waters
top = Topology.from_pdb("receptor_H_solvated.pdb")
top = insert_molecule_and_remove_clashes(top, ligand)

# Save the TOP opbject to a json file to load easier in the future
with open("topology.json", "w") as f:
    print(top.to_json(), file=f)

In [ ]:
sage_ff14sb = ForceField("openff-2.1.0.offxml", "opc-1.0.1.offxml", "ff14sb_off_impropers_0.0.3.offxml")

In [ ]:
#THIS WILL (potentially) BE VERY USEFUL CLASS, see https://docs.openforcefield.org/projects/interchange/en/stable/using/output.html
interchange = sage_ff14sb.create_interchange(top)

In [ ]:
#Write a pdb of the system
interchange.to_pdb("system.pdb")

In [ ]:
#Write AMBER Files using INTERCHANGE
interchange.to_prmtop("system.prmtop")
interchange.to_inpcrd("system.inpcrd")

In [ ]:
#Write GROMACS file using INTERCHANGE
interchange.to_gro("system.gro")
interchange.to_top("system.top")

In [ ]:
run_for_steps(make_openmm_simulation(interchange), 10000)

In [ ]:
#MOST SIMULATIONS STARTED FROM THE AMBER FILES ARE GOING NAN

# from openmm.app import *
# from openmm import *
# from openmm.unit import *
# from sys import stdout

# prmtop = AmberPrmtopFile('system.prmtop')
# inpcrd = AmberInpcrdFile('system.inpcrd')

In [ ]:
# system = prmtop.createSystem()
# integrator = LangevinMiddleIntegrator(300*kelvin, 1/picosecond, 0.001*picoseconds)
# simulation = Simulation(prmtop.topology, system, integrator)
# simulation.context.setPositions(inpcrd.positions)
# if inpcrd.boxVectors is not None:
#     simulation.context.setPeriodicBoxVectors(*inpcrd.boxVectors)
# simulation.minimizeEnergy()
# simulation.reporters.append(PDBReporter('output.pdb', 1000))
# simulation.reporters.append(StateDataReporter(stdout, 1000, step=True,
#         potentialEnergy=True, temperature=True))
# simulation.step(10000)

In [ ]:
# #u = mda.Universe('system.prmtop', topology_format='PARM7')
# import parmed as pmd

# fmt = pmd.amber.AmberFormat('system.prmtop')
# print(fmt.parm_data['POINTERS'][0]) # number of atoms
# print(len(fmt.parm_data['ATOM_NAME']))
# print(len(fmt.parm_data['RADII']))